In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [55]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
train.drop(["comment_text","id"],axis=1).value_counts()

toxic  severe_toxic  obscene  threat  insult  identity_hate
0      0             0        0       0       0                143346
1      0             0        0       0       0                  5666
                     1        0       1       0                  3800
                                      0       0                  1758
                     0        0       1       0                  1215
       1             1        0       1       0                   989
       0             1        0       1       1                   618
0      0             1        0       0       0                   317
                     0        0       1       0                   301
1      1             1        0       1       1                   265
0      0             1        0       1       0                   181
1      1             1        0       0       0                   158
       0             0        0       0       1                   136
                              

In [56]:
import transformers
train.isna().sum(axis=0)

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [57]:
train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [78]:
#from keras.preprocessing.text import Tokenizer
#MAX_NB_WORDS=None #здесь указать максимальное количество сохраняемых слов. Например, первый 1000 самых частых слов
#tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='#$%&()*+-/:;<=>@[\\]^{|}~\t\n,.!"')

# заменяем слова на их индексы в нашем словаре
#X_train = tokenizer.texts_to_sequences(train)
#X_test = tokenizer.texts_to_sequences(test)

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True)
    
#tokenized_datasets = train.map(tokenize_function, batched=True)
tokenised_train = tokenizer(train["comment_text"][0], padding="max_length", truncation=True, return_token_type_ids= False)


In [109]:
s = "hee he l worlde"
tokenizer(s)

{'input_ids': [101, 1119, 1162, 1119, 181, 1362, 1162, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [107]:
tokenizer.decode(tokenizer(s)['input_ids'])

'[CLS] hel he l world [SEP]'

In [84]:
tokenizer.decode(tokenised_train["input_ids"], skip_special_tokens=True)

"Explanation Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now. 89. 205. 38. 27"

In [71]:
train["comment_text"][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [75]:
pd.Series(tokenised_train['input_ids']).value_counts()

0        438
119        5
1103       3
112        3
1116       3
        ... 
16762      1
117        1
1198       1
16409      1
8354       1
Length: 63, dtype: int64

In [76]:
pd.Series(tokenised_train['input_ids']).__len__()

512

In [73]:
pd.Series(tokenised_train['input_ids']).value_counts()

0        444
1012       5
1005       3
1996       3
1056       2
1045       2
16327      1
1998       1
5444       1
2012       1
2047       1
2259       1
14421      1
6904       1
2278       1
102        1
2676       1
4229       1
3531       1
6486       1
2123       1
6366       1
2044       1
2013       1
2831       1
3931       1
2144       1
1049       1
3394       1
2085       1
23561      1
101        1
3806       1
5470       1
2339       1
10086      1
2015       1
2081       1
2104       1
2026       1
5310       1
18442      1
13076      1
12392      1
2050       1
2020       1
7526       1
16407      1
1029       1
2027       1
4694       1
3158       1
9305       1
22556      1
1010       1
2074       1
8503       1
2006       1
2070       1
dtype: int64

In [32]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("We are very happy to show you the 🤗 Transformers library.")

d:\Coding\Python\.venvs\python3.11-ML\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
d:\Coding\Python\.venvs\python3.11-ML\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Chaun\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` envi

[{'label': 'POSITIVE', 'score': 0.9997795224189758}]